In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2 # opencv
from mtcnn.mtcnn import MTCNN
from matplotlib import pyplot as plt
from keras.models import load_model
from PIL import Image
import os
from keras import backend
from tensorflow.keras.models import load_model
import tensorflow as tf
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
import glob
import cv2
import numpy as np
import tensorflow as tf

In [2]:
def extract_face(filename, required_size=(160, 160)):
    # load image from file
    image = Image.open(filename)
    # convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    pixels = np.asarray(image)
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    #for i in range(len(results)):
    # extract the bounding box from the first face
    for i in range(len(results)):
        x1, y1, width, height = results[i]['box']
        # deal with negative pixel index
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + width, y1 + height
        # extract the face
        face = pixels[y1:y2, x1:x2]
        # resize pixels to the model size
        image = Image.fromarray(face)
        image = image.resize(required_size)
        face_array = np.asarray(image)
        return face_array



In [3]:
def load_face(dir):
    faces = list()
    # enumerate files
    for filename in os.listdir(dir):
        path = dir + filename
        face = extract_face(path)
        faces.append(face)
    return faces

def load_dataset(dir):
    # list for faces and labels
    X, y = list(), list()
    for subdir in os.listdir(dir):
        path = dir + subdir + '/'
        faces = load_face(path)
        labels = [subdir for i in range(len(faces))]
        print("loaded %d sample for class: %s" % (len(faces),subdir) ) # print progress
        X.extend(faces)
        y.extend(labels)
    return np.asarray(X), np.asarray(y)

    โหลด dataset จากไฟล์ datas 

In [4]:
trainX, trainy = load_dataset('./datas/train/')
print(trainX.shape, trainy.shape)
# load test dataset
testX, testy = load_dataset('./datas/val/')
print(testX.shape, testy.shape)

loaded 25 sample for class: Alameen
loaded 40 sample for class: Apinun
loaded 40 sample for class: Arim
loaded 40 sample for class: Chanchanok
loaded 40 sample for class: Chareef
loaded 40 sample for class: Jantapa
loaded 40 sample for class: Jetnipat
loaded 40 sample for class: Kanda
loaded 40 sample for class: Ketsiree
loaded 40 sample for class: Nanfa
loaded 40 sample for class: Napadsakorn
loaded 40 sample for class: Naslimin
loaded 40 sample for class: Natthapon
loaded 40 sample for class: Natthawut
loaded 40 sample for class: Nattida
loaded 40 sample for class: Nichakan
loaded 40 sample for class: Pajaree
loaded 40 sample for class: Panjira
loaded 40 sample for class: Pattanun
loaded 40 sample for class: Phimwadi
loaded 40 sample for class: Phiraphon
loaded 40 sample for class: Ruchikon
loaded 40 sample for class: Santi
loaded 40 sample for class: Santiphap
loaded 40 sample for class: Sarawadee
loaded 40 sample for class: Sarayut
loaded 40 sample for class: Sorawit
loaded 40 samp

    บีบอัดข้อมูล dataset จะได้ file npz

In [5]:
np.savez_compressed('faces-dataset.npz', trainX, trainy, testX, testy)


load model

In [6]:
facenet_model = load_model('./facenet_keras.h5')

In [7]:
data = np.load('faces-dataset.npz',allow_pickle=True)

In [8]:
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']

In [9]:
print('Loaded: ', trainX.shape, trainy.shape, testX.shape, testy.shape)

Loaded:  (1373, 160, 160, 3) (1373,) (317, 160, 160, 3) (317,)


ทำ embedding ใบหน้า 

In [10]:
def get_embedding(model, face):
    # scale pixel values
    face = face.astype('float32')
    # standardization
    mean, std = face.mean(), face.std()
    face = (face-mean)/std
    # transfer face into one sample (3 dimension to 4 dimension)
    sample = np.expand_dims(face, axis=0)
    # make prediction to get embedding
    yhat = model.predict(sample)
    return yhat[0]

emdTrainX = list()
for face in trainX:
    emd = get_embedding(facenet_model, face)    
    emdTrainX.append(emd)    
emdTrainX = np.asarray(emdTrainX)
print(emdTrainX.shape)


emdTestX = list()
for face in testX:
    emd = get_embedding(facenet_model, face)
    emdTestX.append(emd)
emdTestX = np.asarray(emdTestX)
print(emdTestX.shape)
np.savez_compressed('faces-embeddings.npz', emdTrainX, trainy, emdTestX, testy) 
print("save face-embeddings success")

(1373, 128)
(317, 128)
save face-embeddings success


tarin and predict 

In [11]:
in_encoder = Normalizer()
emdTrainX_norm = in_encoder.transform(emdTrainX)
emdTestX_norm = in_encoder.transform(emdTestX)

out_encoder = LabelEncoder()
out_encoder.fit(trainy)
trainy_enc = out_encoder.transform(trainy)
testy_enc = out_encoder.transform(testy)

model = SVC(kernel='linear', probability=True)
pred= model.fit(emdTrainX_norm, trainy_enc)

yhat_train = model.predict(emdTrainX_norm)
yhat_test = model.predict(emdTestX_norm)

score_train = accuracy_score(trainy_enc, yhat_train)
score_test = accuracy_score(testy_enc, yhat_test)

print('Accuracy: train=%.3f, test=%.3f' % (score_train*100, score_test*100))

Accuracy: train=100.000, test=100.000


In [12]:
print(yhat_test)


[ 0  0  0  0  0  1  1  1  1  1  1  1  1  2  2  2  2  2  2  2  2  2  2  3
  3  3  3  3  3  3  3  3  3  4  4  4  4  4  4  4  4  4  5  5  5  5  5  5
  5  5  5  5  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  7  7  8  8
  8  8  8  8  8  8  8  8  9  9  9  9  9  9  9  9  9  9 10 10 10 10 10 10
 10 10 10 11 11 11 11 11 11 11 12 12 12 12 12 12 12 12 12 12 13 13 13 13
 13 13 13 13 13 13 14 14 14 14 14 14 14 14 14 14 15 15 15 15 15 15 15 15
 15 15 16 16 16 16 16 16 16 16 16 16 17 17 17 17 17 17 17 17 17 17 18 18
 18 18 18 18 18 18 19 19 19 19 19 19 19 19 19 19 20 20 20 20 20 20 20 20
 20 21 21 21 21 21 21 21 22 22 22 22 22 22 22 22 22 22 23 23 23 23 23 23
 23 23 23 23 24 24 24 24 24 24 24 24 24 24 25 25 25 25 25 25 25 25 25 25
 26 26 26 26 26 26 26 26 26 27 27 27 27 27 27 27 27 27 28 28 28 28 28 28
 28 28 28 28 29 29 29 29 29 29 29 29 29 30 30 30 30 30 30 31 31 31 31 31
 31 31 31 31 31 32 32 32 32 32 33 33 33 33 33 33 33 33 33 33 34 34 34 34
 34 34 34 34 34]


In [13]:
print(score_test)

1.0


In [14]:
from sklearn.metrics import confusion_matrix
confusion_matrix(testy_enc, yhat_test)

array([[ 5,  0,  0, ...,  0,  0,  0],
       [ 0,  8,  0, ...,  0,  0,  0],
       [ 0,  0, 10, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  5,  0,  0],
       [ 0,  0,  0, ...,  0, 10,  0],
       [ 0,  0,  0, ...,  0,  0,  9]], dtype=int64)

In [15]:
image = Image.open('./datas/val/Chareef/3.jpg')
image = image.convert('RGB')
pixels = np.asarray(image)
detector = MTCNN()
results = detector.detect_faces(pixels)
for i in range(len(results)):
    x1, y1, width, height = results[i]['box']
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    face = pixels[y1:y2, x1:x2]
    image = Image.fromarray(face)
    image = image.resize((160,160))
    face_array = np.asarray(image)
    cv2.imwrite('./data/img_{}.jpg'.format(i),face_array)
    
    
data = list()
for i in glob.glob('data/*.jpg', recursive=True):
    data.append(cv2.imread(i))
emdTestX = list()
for face in data:
    emd = get_embedding(facenet_model, face)
    emdTestX.append(emd)
emdTestX = np.asarray(emdTestX)
emdTestX_norm = in_encoder.transform(emdTestX)
yhat_test = model.predict(emdTestX_norm)
print(yhat_test)
predict_name = out_encoder.inverse_transform(yhat_test)
print(predict_name)


[4]
['Chareef']


In [16]:
print(yhat_test[0])

4


In [17]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from matplotlib import pyplot

yhat_prob = model.predict_proba(emdTestX_norm)
class_index = yhat_test[0]
class_probability = yhat_prob[0,class_index] * 100
predict_names = out_encoder.inverse_transform(yhat_test)
print('Predicted: %s (%.3f)' % (predict_names[0], class_probability))

Predicted: Chareef (61.551)


In [18]:
print(yhat_test[0])

4


In [19]:
import pandas as pd
df = pd.read_csv('score.csv')
df

,no,id,name,score
0,1,6035512001,Alameen,NaN
1,2,6035512002,Apinun,NaN
2,3,6035512003,Arim,NaN
3,4,6035512004,Chanchanok,NaN
4,5,6035512005,Chareef,NaN
5,6,6035512006,Jantapa,NaN
6,7,6035512007,Jetnipat,NaN
7,8,6035512008,Kanda,NaN
8,9,6035512009,Ketsiree,NaN
9,10,6035512010,Nanfa,NaN


In [20]:

for i in range(len(yhat_test)):
    print(yhat_test[i])
    predict_name = out_encoder.inverse_transform(yhat_test)
    #df.at[i,'name'] = predict_name[i]
    df.at[yhat_test[i],'score'] = 1

4


In [21]:
df

,no,id,name,score
0,1,6035512001,Alameen,NaN
1,2,6035512002,Apinun,NaN
2,3,6035512003,Arim,NaN
3,4,6035512004,Chanchanok,NaN
4,5,6035512005,Chareef,1.0
5,6,6035512006,Jantapa,NaN
6,7,6035512007,Jetnipat,NaN
7,8,6035512008,Kanda,NaN
8,9,6035512009,Ketsiree,NaN
9,10,6035512010,Nanfa,NaN
